# Aula6 - Doc2Query

[Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)

Stage: calculating metrics after expanding texts with queries 

# Organizando o ambiente

In [1]:
import pickle

In [2]:
import json

In [3]:
import pandas as pd

In [4]:
DIRETORIO_TRABALHO = '/home/borela/fontes/deep_learning_em_buscas_unicamp/local/doc2query'

In [5]:
assert os.path.exists(DIRETORIO_TRABALHO), f"Path para {DIRETORIO_TRABALHO} não existe!"

In [6]:
import os

export JVM_PATH=/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so
export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

In [7]:
os.environ['JVM_PATH'] = '/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

In [8]:
from pyserini.search.lucene import LuceneSearcher

/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Baixando os dados e preparando para avaliação 

## Queries

In [9]:
from pyserini.search import get_topics

In [10]:
topics = get_topics('covid-round5')
print(f'{len(topics)} queries total')

50 queries total


In [11]:
topics[50]

{'question': 'what is known about an mRNA vaccine for the SARS-CoV-2 virus?',
 'query': 'mRNA vaccine coronavirus',
 'narrative': 'Looking for studies specifically focusing on mRNA vaccines for COVID-19, including how mRNA vaccines work, why they are promising, and any results from actual clinical studies.'}

## Relevância (qrel) de teste

In [12]:
if not os.path.exists(f'{DIRETORIO_TRABALHO}/test.tsv'):
    !wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
    !mv test.tsv {DIRETORIO_TRABALHO}/

In [13]:
qrel = pd.read_csv(f"{DIRETORIO_TRABALHO}/test.tsv", sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rel"])

In [14]:
qrel.head()

,query,docid,rel
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1


In [15]:
from tqdm import tqdm

In [16]:
qrel.head()

,query,docid,rel
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1


In [17]:
qrel["q0"] = "q0"
qrel_dict = qrel.to_dict(orient="list")

In [18]:
qrel_dict['query'][0], qrel_dict['docid'][0], qrel_dict['rel'][0]

(1, '005b2j4b', 2)

In [19]:
# Run all queries in topics, retrive top 1k for each query
def run_all_queries(file, topics, searcher, num_max_hits=100):
  """
  A função run_all_queries é responsável por realizar todas as consultas armazenadas no dicionário topics utilizando o objeto searcher fornecido e salvar os resultados em um arquivo de texto.
  Usada no notebook da aula 2

  Parâmetros:

  file: caminho do arquivo de saída onde serão salvos os resultados das consultas.
  topics: dicionário contendo as consultas a serem executadas. Cada consulta é representada por uma chave única no dicionário. O valor correspondente a cada chave é um outro dicionário contendo as informações da consulta, como seu título e outras informações relevantes.
  searcher: objeto do tipo Searcher que será utilizado para realizar as consultas.
  num_max_hits: número máximo de documentos relevantes que serão retornados para cada consulta.
  Retorno:

  A função não retorna nenhum valor, mas salva os resultados das consultas no arquivo especificado em file.
  Comentário:

  A função usa a biblioteca tqdm para exibir uma barra de progresso enquanto executa as consultas.
  O número de consultas concluídas é impresso a cada 100 consultas.
  """
  print(f'Running {len(topics)} queries in total')
  with open(file, 'w') as runfile:
    cnt = 0
    # for id in tqdm(topics, desc='Running Queries'):
    for id in topics:
        # print(f'id = {id}')
        query = topics[id]['question']
        # print(f'query = {query}')

        hits = searcher.search(query, num_max_hits)
        for i in range(0, len(hits)):
            _ = runfile.write(f'{id} Q0 {hits[i].docid} {i+1} {hits[i].score:.6f} ComExpansao\n')
            # = runfile.write('{} Q0 {} {} {:.6f} Pyserini\n'.format(id, hits[i].docid, i+1, hits[i].score))
        cnt += 1
        if cnt % 100 == 0:
            print(f'{cnt} queries completed')


# Carregando os dados a serem indexados

In [20]:
with open(f"{DIRETORIO_TRABALHO}/trec-covid-corpus-sem-texto-sem-queries.pickle", "rb") as f:
  corpus_sem_texto_sem_expansao = pickle.load(f)

In [21]:
with open(f"{DIRETORIO_TRABALHO}/trec-covid-corpus-sem-texto.pickle", "rb") as f:
  corpus_sem_texto_com_expansao_10_queries_bleu_1990 = pickle.load(f)

In [22]:
with open(f"{DIRETORIO_TRABALHO}/trec-covid-corpus-com-texto-com-10queries.pickle", "rb") as f:
  corpus_com_texto_expansao_10_queries_bleu_201 = pickle.load(f)

In [23]:
with open(f"{DIRETORIO_TRABALHO}/trec-covid-corpus-com-texto.pickle", "rb") as f:
  corpus_com_texto_expansao_5_queries_bleu_1990 = pickle.load(f)

# Visualizando os dados a serem usados

In [24]:
dict_corpus = {
    "corpus_com_texto_expansao_5_queries_bleu_1990" : corpus_com_texto_expansao_5_queries_bleu_1990,
    "corpus_sem_texto_com_expansao_10_queries_bleu_1990" : corpus_sem_texto_com_expansao_10_queries_bleu_1990,
    "corpus_com_texto_expansao_10_queries_bleu_201" : corpus_com_texto_expansao_10_queries_bleu_201,
    "corpus_sem_texto_sem_expansao" : corpus_sem_texto_sem_expansao,

}

In [25]:
print(f"Visualizando quantitativos")
print('')
for corpus in dict_corpus:
    print(f"{corpus}")
    print(len(dict_corpus[corpus]))
    print('')

Visualizando quantitativos

corpus_com_texto_expansao_5_queries_bleu_1990
129192

corpus_sem_texto_com_expansao_10_queries_bleu_1990
42140

corpus_com_texto_expansao_10_queries_bleu_211
129192

corpus_sem_texto_sem_expansao
42140



In [26]:
print(f"Visualizando 5o registro")
print('')
for corpus in dict_corpus:
    print(f"{corpus}")
    print(dict_corpus[corpus][5])
    print('')

Visualizando 5o registro

corpus_com_texto_expansao_5_queries_bleu_1990
{'_id': 'zjufx4fo', 'title': 'Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synthesis', 'text': 'Nidovirus subgenomic mRNAs contain a leader sequence derived from the 5′ end of the genome fused to different sequences (‘bodies’) derived from the 3′ end. Their generation involves a unique mechanism of discontinuous subgenomic RNA synthesis that resembles copy-choice RNA recombination. During this process, the nascent RNA strand is transferred from one site in the template to another, during either plus or minus strand synthesis, to yield subgenomic RNA molecules. Central to this process are transcription-regulating sequences (TRSs), which are present at both template sites and ensure the fidelity of strand transfer. Here we present results of a comprehensive co-variation mutagenesis study of equine arteritis virus TRSs, demonstrating that discontinuous RNA synthesis depen

In [27]:
print(f"Visualizando estatísticas")
print('')
for corpus in dict_corpus:
    if "qtd_texto_gerado" in dict_corpus[corpus][0]:
        print(f"{corpus}")
        df_com_texto = pd.DataFrame(dict_corpus[corpus])['qtd_texto_gerado']
        estatisticas = df_com_texto.describe()
        print(estatisticas)    


Visualizando estatísticas

corpus_com_texto_expansao_5_queries_bleu_1990
count    129192.000000
mean          4.072504
std           1.317116
min           1.000000
25%           3.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: qtd_texto_gerado, dtype: float64
corpus_sem_texto_com_expansao_10_queries_bleu_1990
count    42140.000000
mean         7.191552
std          3.028150
min          1.000000
25%          5.000000
50%          8.000000
75%         10.000000
max         10.000000
Name: qtd_texto_gerado, dtype: float64
corpus_com_texto_expansao_10_queries_bleu_211
count    129192.000000
mean          7.433239
std           2.886149
min           1.000000
25%           5.000000
50%           9.000000
75%          10.000000
max          10.000000
Name: qtd_texto_gerado, dtype: float64


# Carregando os dados para geração dos índices

In [28]:
%%time
lista_carga = {}
for corpus in dict_corpus:
    lista_carga[corpus] = []
    for doc in dict_corpus[corpus]:
        if 'texto_gerado' in doc: 
            lista_carga[corpus].append({"id": doc['_id'], "contents":f"{doc['title']} {doc['text']} {doc['texto_gerado']}"})
        else:
            lista_carga[corpus].append({"id": doc['_id'], "contents":f"{doc['title']} "})


CPU times: user 228 ms, sys: 109 ms, total: 337 ms
Wall time: 337 ms


In [29]:
for corpus in dict_corpus:
    print(f" tamanho lista_carga_{corpus}: {len(lista_carga[corpus])}")


 tamanho lista_carga_corpus_com_texto_expansao_5_queries_bleu_1990: 129192
 tamanho lista_carga_corpus_sem_texto_com_expansao_10_queries_bleu_1990: 42140
 tamanho lista_carga_corpus_com_texto_expansao_10_queries_bleu_211: 129192
 tamanho lista_carga_corpus_sem_texto_sem_expansao: 42140


In [30]:
for corpus in dict_corpus:
    print(f"\nCorpus {corpus} Carga: {lista_carga[corpus][5]}")


Corpus corpus_com_texto_expansao_5_queries_bleu_1990 Carga: {'id': 'zjufx4fo', 'contents': 'Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synthesis Nidovirus subgenomic mRNAs contain a leader sequence derived from the 5′ end of the genome fused to different sequences (‘bodies’) derived from the 3′ end. Their generation involves a unique mechanism of discontinuous subgenomic RNA synthesis that resembles copy-choice RNA recombination. During this process, the nascent RNA strand is transferred from one site in the template to another, during either plus or minus strand synthesis, to yield subgenomic RNA molecules. Central to this process are transcription-regulating sequences (TRSs), which are present at both template sites and ensure the fidelity of strand transfer. Here we present results of a comprehensive co-variation mutagenesis study of equine arteritis virus TRSs, demonstrating that discontinuous RNA synthesis depends not only on base 

# Experimentos

In [31]:
from evaluate import load

In [32]:
from pyserini.index import IndexReader

In [33]:
import shutil

In [34]:
trec_eval = load("trec_eval")

In [45]:
num_max_hits = 1000

In [36]:
caminho_indice_trec_covid_com_expansao = f'{DIRETORIO_TRABALHO}/indexes/trec_covid_expanded'

In [37]:
DIRETORIO_RUN = f"{DIRETORIO_TRABALHO}/runs"
CAMINHO_RUN_COM_EXPANSAO = f"{DIRETORIO_RUN}/run-trec-covid-expanded-bm25.txt"

In [40]:
DIRETORIO_INDICE = f"{DIRETORIO_TRABALHO}/indexes/trec-covid-expanded-index/"
CAMINHO_INDICE = f"{DIRETORIO_TRABALHO}/indexes/trec-covid-expanded-index"
DIRETORIO_CARDIN = f"{DIRETORIO_TRABALHO}/cardin/"
CAMINHO_CARDIN = f"{DIRETORIO_TRABALHO}/trec-covid-expanded-cardin/trec-covid-expanded.jsonl"


In [41]:
%%time
if not os.path.exists(DIRETORIO_RUN):
  os.makedirs(DIRETORIO_RUN)
  print('pasta criada!')
else:
  print('pasta já existia!')


pasta já existia!
CPU times: user 128 µs, sys: 40 µs, total: 168 µs
Wall time: 293 µs


In [42]:
%%time
if not os.path.exists(DIRETORIO_INDICE):
  os.makedirs(DIRETORIO_INDICE)
  print('pasta criada!')
else:
  print('pasta já existia!')


pasta criada!
CPU times: user 330 µs, sys: 105 µs, total: 435 µs
Wall time: 334 µs


In [43]:
%%time
if not os.path.exists(DIRETORIO_CARDIN):
  os.makedirs(DIRETORIO_CARDIN)
  print('pasta criada!')
else:
  print('pasta já existia!')


pasta já existia!
CPU times: user 96 µs, sys: 30 µs, total: 126 µs
Wall time: 166 µs


In [44]:
dict_experimentos = {
    "expansão em documentos com texto de até 5 queries (modelo com BLEU 19.9) e sem expansão em sem texto": ['corpus_com_texto_expansao_5_queries_bleu_1990', 'corpus_sem_texto_sem_expansao'],
    "expansão em documentos com texto de até 5 queries (modelo com BLEU 19.9) e com expansão em sem texto": ['corpus_com_texto_expansao_5_queries_bleu_1990', 'corpus_sem_texto_com_expansao_10_queries_bleu_1990'],
    "expansão em documentos com texto de até 5 queries (modelo com BLEU 19.9) e sem documentos sem texto": ['corpus_com_texto_expansao_5_queries_bleu_1990'],
    "expansão em documentos com texto de até 10 queries (modelo com BLEU 20.1) e sem expansão em sem texto": ['corpus_com_texto_expansao_10_queries_bleu_201', 'corpus_sem_texto_sem_expansao'],
    "expansão em documentos com texto de até 10 queries (modelo com BLEU 20.1) e com expansão em sem texto": ['corpus_com_texto_expansao_10_queries_bleu_201', 'corpus_sem_texto_com_expansao_10_queries_bleu_1990'],
    "expansão em documentos com texto de até 10 queries (modelo com BLEU 20.1) e sem documentos sem texto": ['corpus_com_texto_expansao_10_queries_bleu_201'],
}

In [54]:
dict_resultado = {}


for experimento in dict_experimentos:
    # Verificar se o arquivo de índice existe e apagar 
    # Remover arquivos que serão gerados 
    if os.path.exists(DIRETORIO_INDICE):
        shutil.rmtree(DIRETORIO_INDICE)
    if os.path.exists(CAMINHO_RUN_COM_EXPANSAO):
        os.remove(CAMINHO_RUN_COM_EXPANSAO)

    print(f'\nExperimento {experimento}')
    with open(CAMINHO_CARDIN, 'w') as arquivo:
        for corpus in dict_experimentos[experimento]:
            for doc in lista_carga[corpus]:
                # Converte o dicionário em uma string JSON
                linha_json = json.dumps(doc)
                # Escreve a linha JSON no arquivo
                arquivo.write(f'{linha_json}\n')

    # constrói o índice;  > /dev/null 2>&1 para não gerar output
    !python -m pyserini.index.lucene \
            --collection JsonCollection \
            --input {DIRETORIO_CARDIN} \
            --index {DIRETORIO_INDICE} \
            --generator DefaultLuceneDocumentGenerator \
            --threads 9 \
            --storePositions --storeDocvectors \
            --storeRaw  > /dev/null 2>&1 

    # Imprimindo estatísticas do índice
    index_reader = IndexReader(DIRETORIO_INDICE)
    print(index_reader.stats())

    #!mv indexes/trec_covid_expanded {DIRETORIO_TRABALHO}/indexes
    # cria o mecanismo de busca
    searcher = LuceneSearcher(DIRETORIO_INDICE)
    searcher.set_bm25(k1=0.82, b=0.68)  
    run_all_queries(CAMINHO_RUN_COM_EXPANSAO, topics, searcher, num_max_hits)
    ### Calculando métricas
    run = pd.read_csv(f"{CAMINHO_RUN_COM_EXPANSAO}", sep="\s+", 
                    names=["query", "q0", "docid", "rank", "score", "system"])
    print(run.head())
    run = run.to_dict(orient="list")
    results = trec_eval.compute(predictions=[run], references=[qrel_dict])

    # salvando métricas    
    print(f"NDCG@10: {results['NDCG@10']}")
    dict_resultado[experimento] = {
     'ndcg@10': results['NDCG@10'], 'metrics': results}

print('Resultado final')
for experimento in dict_experimentos:
    if experimento in dict_resultado:
        print(dict_resultado[experimento]['ndcg@10'],' - ', experimento)


Experimento expansão em documentos com texto de até 5 queries (modelo com BLEU 19.9) e sem expansão em sem texto
{'total_terms': 22975535, 'documents': 171331, 'non_empty_documents': 171331, 'unique_terms': 233874}
Running 50 queries in total
   query  q0     docid  rank    score       system
0     44  Q0  xfjexm5b     1  12.8608  ComExpansao
1     44  Q0  ihyj9fpi     2  12.4281  ComExpansao
2     44  Q0  5cm6122n     3  11.9008  ComExpansao
3     44  Q0  14x4uqq7     4  11.7633  ComExpansao
4     44  Q0  zwyueevh     5  11.6887  ComExpansao
NDCG@10: 0.6163398516906394

Experimento expansão em documentos com texto de até 5 queries (modelo com BLEU 19.9) e com expansão em sem texto
{'total_terms': 24269645, 'documents': 171332, 'non_empty_documents': 171332, 'unique_terms': 257262}
Running 50 queries in total
   query  q0     docid  rank    score       system
0     44  Q0  xfjexm5b     1  12.7865  ComExpansao
1     44  Q0  ihyj9fpi     2  12.1623  ComExpansao
2     44  Q0  5cm6122n   

# Extras

### Investigando um índice 
Dicas em https://github.com/castorini/pyserini/blob/master/docs/usage-indexreader.md 

In [ ]:
from pyserini.index import IndexReader

# Diretório do índice
caminho_indice = f"{DIRETORIO_TRABALHO}/indexes/trec_covid_expanded"

# Criação do objeto IndexReader
index_reader = IndexReader(caminho_indice)


Total de documentos

In [ ]:

print(index_reader.stats())


{'total_terms': 24724735, 'documents': 129192, 'non_empty_documents': 129192, 'unique_terms': 252425}


Pesquisando

In [ ]:
term = 'cities'

# Look up its document frequency (df) and collection frequency (cf).
# Note, we use the unanalyzed form:
df, cf = index_reader.get_term_counts(term)
print(f'term "{term}": df={df}, cf={cf}')

term "cities": df=3296, cf=6082


In [ ]:
# Note that the keys of get_document_vector() are already analyzed, we set analyzer to be None.
bm25_score_genotyp = index_reader.compute_bm25_term_weight('zjufx4fo', 'genotyp', analyzer=None)
bm25_score_genotype = index_reader.compute_bm25_term_weight('zjufx4fo', 'genotype', analyzer=None)
print(f"bm25_score_genotype: {bm25_score_genotype}, bm25_score_genotyp:{bm25_score_genotyp}")

bm25_score_genotype: 0.0, bm25_score_genotyp:2.7530345916748047
